In [38]:
import pandas as pd
from config import *
from ppp_prediction.metrics.utils import format_metrics

# Supplementary Table

## Supp Table 4

In [22]:
centers = [
    "AZ-TR",
    "AZ-EV1",
    "AZ-EV2",
    "AZTZ-EV",
    "DX-EV",
]

metrics_qt_df = pd.read_feather(RawDataDir / "metrics_qt.feather")
metrics_qt_df["Center"] = pd.Categorical(
    metrics_qt_df["Center"], categories=centers, ordered=True
)
metrics_qt_df["Label"] = pd.Categorical(
    metrics_qt_df["Label"], categories=qts, ordered=False
)
metrics_qt_df

,Model,Label,Center,N,spearmanr,pearsonr,p_adj,pvalue,R2,R2_LCI,R2_UCI,R2 (95% CI),spearmanr_LCI,spearmanr_UCI,spearmanr (95% CI),pearsonr_LCI,pearsonr_UCI,pearsonr (95% CI)
0,ResNet18,AG,AZ-EV1,1474,0.074119,-0.007757,8.067738e-01,7.659982e-01,0.000060,-0.000746,0.000866,"0.000 (-0.001, 0.001)",0.036330,0.096290,"0.074 (0.036, 0.096)",-0.053324,0.040154,"-0.008 (-0.053, 0.040)"
1,ResNet18,ALP,AZ-EV1,1152,0.120766,0.142960,2.376341e-06,9.665502e-07,0.020438,0.003984,0.036891,"0.020 (0.004, 0.037)",0.072613,0.155722,"0.121 (0.073, 0.156)",0.106501,0.187912,"0.143 (0.107, 0.188)"
2,ResNet18,ALT,AZ-EV1,1495,0.260491,0.203048,5.335247e-15,1.123548e-15,0.041228,0.021136,0.061321,"0.041 (0.021, 0.061)",0.246248,0.301909,"0.260 (0.246, 0.302)",0.119427,0.229035,"0.203 (0.119, 0.229)"
3,ResNet18,APTT,AZ-EV1,1495,0.044554,0.036771,1.988947e-01,1.550948e-01,0.001352,-0.002438,0.005142,"0.001 (-0.002, 0.005)",0.016075,0.097595,"0.045 (0.016, 0.098)",-0.009976,0.067058,"0.037 (-0.010, 0.067)"
4,ResNet18,AST,AZ-EV1,1495,-0.007178,0.044670,1.146657e-01,8.403301e-02,0.001995,-0.002606,0.006597,"0.002 (-0.003, 0.007)",-0.035225,0.032541,"-0.007 (-0.035, 0.033)",0.006875,0.082360,"0.045 (0.007, 0.082)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
826,Ummae,Urea,DX-EV,895,0.189554,0.193474,1.176252e-08,3.793449e-09,0.037432,0.012629,0.062235,"0.037 (0.013, 0.062)",0.133096,0.235492,"0.190 (0.133, 0.235)",0.167554,0.252996,"0.193 (0.168, 0.253)"
827,Ummae,WBC,DX-EV,893,0.148479,0.139583,5.568552e-05,2.579895e-05,0.019483,0.001235,0.037732,"0.019 (0.001, 0.038)",0.102851,0.191618,"0.148 (0.103, 0.192)",0.085855,0.168623,"0.140 (0.086, 0.169)"
828,Ummae,Weight,DX-EV,800,0.682028,0.676797,2.055708e-147,1.063724e-148,0.458055,0.406409,0.509700,"0.458 (0.406, 0.510)",0.653147,0.719780,"0.682 (0.653, 0.720)",0.629848,0.738134,"0.677 (0.630, 0.738)"
829,Ummae,eGFR(CKD-EPI),DX-EV,895,0.474502,0.428795,1.041746e-44,1.153317e-45,0.183865,0.137257,0.230473,"0.184 (0.137, 0.230)",0.444578,0.515038,"0.475 (0.445, 0.515)",0.391157,0.480555,"0.429 (0.391, 0.481)"


In [24]:
system_metrics_qt_df_dict = {}
tgt_metric = "R2"
for system, traits in System_traits_map.items():
    for trait in traits:
        if trait not in metrics_qt_df["Label"].values:
            print(f"Trait {trait} not of {system} not founded in current data")
    system_metrics_qt_df_extracted = metrics_qt_df[metrics_qt_df["Label"].isin(traits)]
    system_metrics_qt_df_extracted_summstats = (
        system_metrics_qt_df_extracted.groupby(["Model", "Center"])
        .apply(lambda x: x[tgt_metric].describe()[["50%", "25%", "75%", "count"]])
        .sort_index()
    )
    # median 0.055, 0.022-0.11
    # system_metrics_qt_df_extracted_summstats[f"{tgt_metric}_fmt"] = (
    #     "median "
    #     + system_metrics_qt_df_extracted_summstats["50%"].apply(lambda x: f"{x:.3f}")
    #     + ", "
    #     + system_metrics_qt_df_extracted_summstats["25%"].apply(lambda x: f"{x:.3f}")
    #     + "-"
    #     + system_metrics_qt_df_extracted_summstats["75%"].apply(lambda x: f"{x:.3f}")
    # )
    system_metrics_qt_df_extracted_summstats[f"{tgt_metric}_fmt"] = (
        system_metrics_qt_df_extracted_summstats["50%"].apply(lambda x: f"{x:.3f}")
        + " ("
        + system_metrics_qt_df_extracted_summstats["25%"].apply(lambda x: f"{x:.3f}")
        + " - "
        + system_metrics_qt_df_extracted_summstats["75%"].apply(lambda x: f"{x:.3f}")
        + ", N="
        + system_metrics_qt_df_extracted_summstats["count"].apply(lambda x: f"{x:.0f}")
        + ")"
    )
    system_metrics_qt_df_extracted_summstats = (
        system_metrics_qt_df_extracted_summstats[["count", f"{tgt_metric}_fmt"]]
        .reset_index(drop=False)
        .rename(columns={"count": "N"})
    )
    # format

    system_metrics_qt_df_extracted_summstats_p1 = (
        system_metrics_qt_df_extracted_summstats[
            ["Model", "Center", f"{tgt_metric}_fmt"]
        ]
    )
    system_metrics_qt_df_extracted_summstats_p1.columns = ["Model", "Center", "Value"]
    system_metrics_qt_df_extracted_summstats_p2 = (
        (system_metrics_qt_df_extracted_summstats[["Model", "Center", "N"]])
        .groupby("Center")
        .max()
        .reset_index(drop=False)
    ).rename(columns={"N": "Value"})
    # system_metrics_qt_df_extracted
    system_metrics_qt_df_extracted_summstats_p2["Model"] = "N"

    system_metrics_qt_df_extracted_summstats_concat_data = pd.concat(
        [
            system_metrics_qt_df_extracted_summstats_p1,
            system_metrics_qt_df_extracted_summstats_p2,
        ],
        axis=0,
    )
    system_metrics_qt_df_extracted_summstats_concat_data["System"] = system
    system_metrics_qt_df_dict[system] = (
        system_metrics_qt_df_extracted_summstats_concat_data
    )


system_metrics_qt_df = pd.concat(system_metrics_qt_df_dict.values(), axis=0)
system_metrics_qt_df

Trait EF not of Cardiovascular not founded in current data


,Model,Center,Value,System
0,ResNet18,AZ-TR,"0.429 (0.291 - 0.630, N=4)",Anthropometrics
1,ResNet18,AZ-EV1,"0.443 (0.348 - 0.599, N=4)",Anthropometrics
2,ResNet18,AZ-EV2,"0.508 (0.484 - 0.600, N=4)",Anthropometrics
3,ResNet18,AZTZ-EV,"0.496 (0.406 - 0.620, N=4)",Anthropometrics
4,ResNet18,DX-EV,"0.430 (0.350 - 0.555, N=4)",Anthropometrics
...,...,...,...,...
0,N,AZ-TR,4.0,Renal
1,N,AZ-EV1,4.0,Renal
2,N,AZ-EV2,4.0,Renal
3,N,AZTZ-EV,4.0,Renal


In [25]:
system_metrics_qt_df["Model"] = pd.Categorical(
    system_metrics_qt_df["Model"], categories=["ResNet18", "SwinLarge", "Ummae"]
)
T4 = (
    system_metrics_qt_df.dropna(subset=["Model"]).pivot(
        index="System", columns=["Center", "Model"], values="Value"
    )
    # .swaplevel(axis=1)
    .sort_index(axis=1)
)
T4.to_excel(TableDir / "SupplementaryTable4.xlsx")
T4

Center                                 AZ-TR                               \
Model                               ResNet18                    SwinLarge   
System                                                                      
Anthropometrics   0.429 (0.291 - 0.630, N=4)   0.491 (0.353 - 0.671, N=4)   
Cardiovascular   0.004 (0.002 - 0.015, N=15)  0.025 (0.002 - 0.045, N=15)   
Digestive        0.011 (0.003 - 0.043, N=10)  0.039 (0.008 - 0.049, N=10)   
Electrolytes      0.011 (0.007 - 0.014, N=3)   0.023 (0.012 - 0.024, N=3)   
Endocrine        0.003 (0.002 - 0.012, N=10)  0.005 (0.001 - 0.037, N=10)   
Hematology       0.022 (0.004 - 0.165, N=11)  0.020 (0.015 - 0.198, N=11)   
Immune            0.010 (0.001 - 0.023, N=8)   0.012 (0.007 - 0.033, N=8)   
Renal             0.069 (0.023 - 0.146, N=4)   0.078 (0.030 - 0.155, N=4)   

Center                                                             AZ-EV1  \
Model                                  Ummae                     ResNet18   
System                                                                      
Anthropometrics   0.493 (0.361 - 0.666, N=4)   0.443 (0.348 - 0.599, N=4)   
Cardiovascular   0.037 (0.010 - 0.051, N=15)  0.007 (0.002 - 0.020, N=15)   
Digestive        0.035 (0.013 - 0.071, N=10)  0.014 (0.003 - 0.021, N=10)   
Electrolytes      0.020 (0.013 - 0.036, N=3)   0.002 (0.001 - 0.003, N=3)   
Endocrine        0.024 (0.007 - 0.055, N=10)   0.004 (0.002 - 0.012, N=9)   
Hematology       0.036 (0.022 - 0.208, N=11)  0.006 (0.003 - 0.113, N=11)   
Immune            0.034 (0.022 - 0.073, N=8)   0.017 (0.002 - 0.022, N=8)   
Renal             0.081 (0.038 - 0.161, N=4)   0.082 (0.015 - 0.159, N=4)   

Center                                                                     \
Model                              SwinLarge                        Ummae   
System                                                                      
Anthropometrics   0.465 (0.379 - 0.609, N=4)   0.471 (0.377 - 0.620, N=4)   
Cardiovascular   0.018 (0.004 - 0.041, N=15)  0.026 (0.013 - 0.042, N=15)   
Digestive        0.024 (0.019 - 0.033, N=10)  0.036 (0.020 - 0.041, N=10)   
Electrolytes      0.003 (0.002 - 0.003, N=3)   0.006 (0.003 - 0.007, N=3)   
Endocrine         0.009 (0.001 - 0.035, N=9)   0.023 (0.003 - 0.047, N=9)   
Hematology       0.014 (0.007 - 0.130, N=11)  0.017 (0.007 - 0.135, N=11)   
Immune            0.030 (0.017 - 0.040, N=8)   0.049 (0.041 - 0.058, N=8)   
Renal             0.088 (0.032 - 0.155, N=4)   0.081 (0.024 - 0.155, N=4)   

Center                                AZ-EV2                               \
Model                               ResNet18                    SwinLarge   
System                                                                      
Anthropometrics   0.508 (0.484 - 0.600, N=4)   0.526 (0.511 - 0.609, N=4)   
Cardiovascular   0.007 (0.001 - 0.014, N=15)  0.012 (0.002 - 0.022, N=15)   
Digestive        0.008 (0.002 - 0.017, N=10)  0.012 (0.005 - 0.021, N=10)   
Electrolytes      0.000 (0.000 - 0.000, N=3)   0.001 (0.001 - 0.004, N=3)   
Endocrine         0.001 (0.000 - 0.007, N=9)   0.007 (0.000 - 0.017, N=9)   
Hematology       0.009 (0.002 - 0.112, N=11)  0.006 (0.002 - 0.124, N=11)   
Immune            0.011 (0.001 - 0.015, N=8)   0.018 (0.009 - 0.028, N=8)   
Renal             0.069 (0.023 - 0.128, N=4)   0.068 (0.034 - 0.115, N=4)   

Center                                                            AZTZ-EV  \
Model                                  Ummae                     ResNet18   
System                                                                      
Anthropometrics   0.521 (0.493 - 0.615, N=4)   0.496 (0.406 - 0.620, N=4)   
Cardiovascular   0.023 (0.009 - 0.033, N=15)  0.001 (0.000 - 0.009, N=13)   
Digestive        0.022 (0.009 - 0.033, N=10)   0.005 (0.001 - 0.050, N=9)   
Electrolytes      0.001 (0.000 - 0.001, N=3)       nan (nan - nan, N=nan)   
Endocrine         0.003 (0.002 - 0.017, N=9)   0.004 (0.002 - 0.010, 

## Supp Table 5

In [26]:
tgt_metric = "R2"
table_data = metrics_qt_df.copy()
table_data[f"{tgt_metric} (95% CI)"] = (
    table_data[f"{tgt_metric} (95% CI)"]
    + ", adjusted P-Value = "
    + table_data[f"p_adj"].apply(lambda x: f"{x:.2e}")
)
T5_data_p1 = table_data[["Model", "Label", "Center", "N"]].rename(
    columns={"N": "Value"}
)

T5_data_p2 = table_data[
    [
        "Model",
        "Label",
        "Center",
        f"{tgt_metric} (95% CI)",
    ]
].rename(columns={f"{tgt_metric} (95% CI)": "Value"})

T5_data_p1 = T5_data_p1.groupby(["Label", "Center"]).max().reset_index(drop=False)
T5_data_p1.drop(columns=["Model"], inplace=True)
T5_data_p1["Model"] = "N"


T5_data = pd.concat([T5_data_p1, T5_data_p2])
T5_data["Model"] = pd.Categorical(
    T5_data["Model"], ["ResNet18", "SwinLarge", "Ummae", "N"], ordered=True
)
T5 = (
    T5_data.pivot(index="Label", columns=["Model", "Center"], values=["Value"])
    .swaplevel(axis=1)
    .sort_index(axis=1)
)
T5.to_excel(TableDir / "SupplementaryTable5.xlsx")
T5

Value  \
Center                                              AZ-TR   
Model                                            ResNet18   
Label                                                       
AG      0.018 (-0.006, 0.043), adjusted P-Value = 5.80...   
ALP     0.003 (-0.007, 0.012), adjusted P-Value = 3.32...   
ALT     0.058 (0.016, 0.100), adjusted P-Value = 2.99e-07   
APTT    0.000 (-0.003, 0.003), adjusted P-Value = 7.92...   
AST     0.002 (-0.007, 0.011), adjusted P-Value = 3.63...   
...                                                   ...   
hs-CRP  0.001 (-0.004, 0.006), adjusted P-Value = 6.14...   
hsTnI   0.000 (-0.001, 0.001), adjusted P-Value = 9.45...   
nonHDL  0.007 (-0.008, 0.022), adjusted P-Value = 9.40...   
sdLDL   0.004 (-0.009, 0.017), adjusted P-Value = 2.67...   
Age     0.854 (0.829, 0.878), adjusted P-Value = 0.00e+00   

                                                           \
Center                                                      
Model                                           SwinLarge   
Label                                                       
AG      0.005 (-0.008, 0.018), adjusted P-Value = 1.60...   
ALP     0.005 (-0.008, 0.017), adjusted P-Value = 1.88...   
ALT     0.049 (0.010, 0.088), adjusted P-Value = 2.53e-06   
APTT    0.009 (-0.008, 0.026), adjusted P-Value = 5.73...   
AST     0.000 (-0.002, 0.003), adjusted P-Value = 8.06...   
...                                                   ...   
hs-CRP  0.002 (-0.006, 0.010), adjusted P-Value = 4.21...   
hsTnI   0.002 (-0.006, 0.009), adjusted P-Value = 4.49...   
nonHDL  0.025 (-0.003, 0.053), adjusted P-Value = 9.49...   
sdLDL   0.034 (-0.002, 0.070), adjusted P-Value = 4.20...   
Age     0.870 (0.848, 0.892), adjusted P-Value = 0.00e+00   

                                                                  \
Center                                                             
Model                                               Ummae      N   
Label                                                              
AG      0.001 (-0.004, 0.005), adjusted P-Value = 6.51...  462.0   
ALP     0.010 (-0.008, 0.028), adjusted P-Value = 4.69...  462.0   
ALT     0.075 (0.028, 0.122), adjusted P-Value = 3.16e-09  462.0   
APTT    0.004 (-0.008, 0.016), adjusted P-Value = 1.97...  471.0   
AST     0.003 (-0.007, 0.014), adjusted P-Value = 2.63...  462.0   
...                                                   ...    ...   
hs-CRP  0.011 (-0.008, 0.029), adjusted P-Value = 4.26...  455.0   
hsTnI   0.006 (-0.008, 0.019), adjusted P-Value = 1.37...  473.0   
nonHDL  0.038 (0.004, 0.072), adjusted P-Value = 3.79e-05  473.0   
sdLDL   0.037 (-0.001, 0.074), adjusted P-Value = 2.65...  387.0   
Age     0.873 (0.852, 0.895), adjusted P-Value = 0.00e+00  480.0   

                                                           \
Center                                             AZ-EV1   
Model                                            ResNet18   
Label                                                       
AG      0.000 (-0.001, 0.001), adjusted P-Value = 8.07...   
ALP     0.020 (0.004, 0.037), adjusted P-Value = 2.38e-06   
ALT     0.041 (0.021, 0.061), adjusted P-Value = 5.34e-15   
APTT    0.001 (-0.002, 0.005), adjusted P-Value = 1.99...   
AST     0.002 (-0.003, 0.007), adjusted P-Value = 1.15...   
...                                                   ...   
hs-CRP  0.000 (-0.002, 0.003), adjusted P-Value = 5.61...   
hsTnI   0.001 (-0.002, 0.004), adjusted P-Value = 3.39...   
nonHDL  0.005 (-0.002, 0.013), adjusted P-Value = 8.83...   
sdLDL   0.008 (-0.003, 0.019), adjusted P-Value = 8.83...   
Age     0.861 (0.848, 0.874), adjusted P-Value = 0.00e+00   

                                                           \
Center                                                      
Model                                           SwinLarge   
Label                                                       
AG      0.000 (-0.002, 0.002), adjusted

## Supp Table 6

In [28]:
relative_df = pd.read_feather(RawDataDir / f"RelativeImportance_Ummae_concat.feather")
adjust_r2_pvalue = pd.read_csv(RawDataDir / "adjusted_ummae_r2.csv")
adjust_r2_pvalue

,Model,Label,Center,pearsonr,spearmanr,N,R2,p_adj,pvalue
0,Ummae,HbA1C%,AZ-EV1,0.352716,0.322341,1429,0.124409,2.981030e-44,5.229877e-46
1,Ummae,TG,AZ-EV1,0.224667,0.106085,1450,0.050475,4.937820e-17,1.732568e-18
2,Ummae,Glu,AZ-EV1,0.189534,0.159546,1454,0.035923,3.612770e-12,1.901458e-13
3,Ummae,Hb,AZ-EV1,0.164079,0.163665,1451,0.026922,3.457753e-09,2.426493e-10
4,Ummae,LYM,AZ-EV1,0.151238,0.161752,1451,0.022873,6.551018e-08,5.746507e-09
5,Ummae,HDLC,AZ-EV1,0.144161,0.098044,1450,0.020782,2.814658e-07,2.962798e-08
6,Ummae,nonHDL,AZ-EV1,0.141756,0.121235,1406,0.020095,6.562744e-07,8.059510e-08
7,Ummae,sdLDL,AZ-EV1,0.141688,0.127835,1001,0.020076,2.882787e-05,6.069024e-06
8,Ummae,FFA,AZ-EV1,0.136201,0.114612,1000,0.018551,6.156390e-05,1.404089e-05
9,Ummae,WBC,AZ-EV1,0.127340,0.130535,1451,0.016216,7.291703e-06,1.023397e-06


In [30]:
adjust_r2_pvalue_simple = adjust_r2_pvalue[
    ["Label", "R2", "pearsonr", "spearmanr", "N", "p_adj"]
].rename(
    columns={"R2": "R2_adj", "pearsonr": "pearsonr_adj", "spearmanr": "spearmanr_adj"}
)
adjust_r2_pvalue_simple["Label"] = adjust_r2_pvalue_simple["Label"].replace(
    {"HbA1C%": "HbA1C"}
)
adjust_r2_pvalue_simple.set_index("Label", inplace=True)
adjust_r2_pvalue_simple
relative_df
T6 = (
    relative_df[["Omics", "Importance", "Label"]]
    .pivot(index="Label", columns="Omics", values="Importance")
    .merge(adjust_r2_pvalue_simple, left_index=True, right_index=True)
)
T6.to_excel(TableDir / "SupplementaryTable6.xlsx")
T6

,Age,BMI,Height,Sex,Ummae,Weight,R2_adj,pearsonr_adj,spearmanr_adj,N,p_adj
Label,,,,,,,,,,,
ALP,0.032177,0.028095,0.178330,0.178972,0.509121,0.073305,0.005855,0.076520,0.056294,1124,2.225195e-02
Alb,0.125312,0.068891,0.066045,0.009068,0.657914,0.072772,0.013172,0.114768,0.111242,1114,4.163797e-04
BNP,0.169962,0.047588,0.016393,0.004900,0.711648,0.049507,0.007007,0.083708,-0.143195,1442,3.715688e-03
D-Bil,0.045390,0.052890,0.104536,0.137129,0.574015,0.086040,0.011095,0.105335,0.077518,1450,2.113246e-04
FFA,0.010357,0.046903,0.108111,0.184545,0.612109,0.037975,0.018551,0.136201,0.114612,1000,6.156390e-05
GGT,0.110923,0.045321,0.038161,0.058549,0.675772,0.071275,0.006268,0.079168,0.006492,1124,1.780671e-02
Glu,0.033342,0.005810,0.002301,0.000676,0.953053,0.004818,0.035923,0.189534,0.159546,1454,3.612770e-12
HCT,0.074094,0.042356,0.105803,0.231165,0.437613,0.108968,0.012389,0.111304,0.138568,1451,8.199062e-05
HDLC,0.070298,0.083779,0.088704,0.143441,0.457122,0.156657,0.020782,0.144161,0.098044,1450,2.814658e-07


## Supp Table 7 

In [31]:
compare_res_df_fdr = pd.read_feather(RawDataDir / "Genetics_120_results.feather")
T7 = (
    compare_res_df_fdr[["Traits", "Feature", "N", "R2 (95% CI)", "p_adj", "pvalue"]]
    .rename(columns={"Feature": "Predictor"})
    .sort_values(["Traits", "Predictor"])
)
T7.to_excel(TableDir / "SupplementaryTable7.xlsx", index=False)
T7

,Traits,Predictor,N,R2 (95% CI),p_adj,pvalue
657,ALP,Face,40,"0.001 (-0.013, 0.015)",0.924920,0.881625
619,ALP,PGS000670,40,"0.001 (-0.021, 0.024)",0.878821,0.811507
654,ALP,PGS000815,40,"0.001 (-0.014, 0.015)",0.924920,0.878948
368,ALP,PGS001939,40,"0.023 (-0.063, 0.110)",0.510409,0.340996
539,ALP,PGS002157,40,"0.005 (-0.035, 0.044)",0.792406,0.674388
...,...,...,...,...,...,...
509,eGFR(CKD-EPI),PGS004089,111,"0.006 (-0.022, 0.034)",0.592135,0.415754
515,eGFR(CKD-EPI),PGS004113,111,"0.006 (-0.022, 0.033)",0.603169,0.429347
550,eGFR(CKD-EPI),PGS004129,111,"0.004 (-0.020, 0.028)",0.660988,0.497850
470,eGFR(CKD-EPI),PGS004143,111,"0.010 (-0.027, 0.047)",0.453732,0.285798


## Supp Table 8

In [32]:
RelativeDataDir = RawDataDir / "RelativeImportance_Cardiovascular_full_Genetics.csv"

relim_df = pd.read_csv(RelativeDataDir)
T_relim_df = (
    relim_df[["Traits", "Omics", "Importance"]]
    .pivot(index="Traits", columns="Omics", values="Importance")
    .sort_values("Face", ascending=False)
)

T8 = T_relim_df.sort_index()
T8.to_excel(TableDir / "SupplementaryTable8.xlsx", index=True)
T8

Omics,Face,Genetics
Traits,,
ALP,0.020635,0.979365
ALT,0.825379,0.174621
AST,0.031179,0.968821
Alb,0.738711,0.261289
BMI,0.821486,0.178514
Cr,0.906512,0.093488
GGT,0.640677,0.359323
Glu,0.511985,0.488015
HCT,0.880988,0.119012


## Supp Table 9

In [9]:
assoc_data_dict = pd.read_pickle(RawDataDir / "Assoc_data_dict_stenosis.pkl")

res_real = assoc_data_dict["Real"]
res_ummae = assoc_data_dict["Ummae"]

In [18]:
from ppp_prediction.metrics.utils import format_metrics

res_ummae_simple = res_ummae[["x", "OR", "OR_UCI", "OR_LCI", "pvalue"]]
# res_ummae_simple['OR'] = res_ummae_simple['OR']* -1
# res_ummae_simple['OR_UCI'] = res_ummae_simple['OR_UCI'] * -1
# res_ummae_simple['OR_LCI'] = res_ummae_simple['OR_LCI'] * -1

res_ummae_simple["x"] = res_ummae_simple["x"].str.split("_").str[0]


res_real_simple = res_real[["x", "OR", "OR_UCI", "OR_LCI", "pvalue"]]


T9 = res_ummae_simple.merge(
    res_real_simple, on=["x"], suffixes=("_Facial", "_Real"), how="outer"
).sort_values("x")


for suffix in ["Real", "Facial"]:

    T9[f"OR {suffix} (95% CI)"] = format_metrics(
        T9[f"OR_{suffix}"],
        x_lci=T9[f"OR_LCI_{suffix}"],
        x_uci=T9[f"OR_UCI_{suffix}"],
        round=3,
        data=T9,
    )

T9 = (
    T9[["x", "OR Real (95% CI)", "pvalue_Real", "OR Facial (95% CI)", "pvalue_Facial"]]
    # .query("x in @cardio_vascular")
    .rename(
        columns={
            "x": "Blood biomarkers",
            "pvalue_Real": "P (Lab measurment)",
            "pvalue_Facial": "P (Facial prediction)",
            "OR Facial (95% CI)": "OR (Facial prediction)",
            "OR Real (95% CI)": "OR (Lab measurement)",
        }
    ).reset_index(drop=True)
)
T9.to_excel(TableDir / "SupplementaryTable9.xlsx", index=False)
T9

/tmp/ipykernel_1838233/3412418913.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_ummae_simple["x"] = res_ummae_simple["x"].str.split("_").str[0]


,Blood biomarkers,OR (Lab measurement),P (Lab measurment),OR (Facial prediction),P (Facial prediction)
0,BNP,"1.121 (0.980, 1.284)",9.642163e-02,"0.744 (0.639, 0.866)",1.413714e-04
1,CK,"0.883 (0.778, 1.004)",5.684975e-02,"0.971 (0.777, 1.212)",7.929297e-01
2,CK-MB,"0.937 (0.745, 1.179)",5.808989e-01,"1.009 (0.856, 1.188)",9.173005e-01
3,D-Dimer,"1.038 (0.806, 1.337)",7.740715e-01,"0.723 (0.623, 0.839)",1.989828e-05
4,HDLC,"0.856 (0.749, 0.978)",2.232867e-02,"0.705 (0.579, 0.858)",4.789082e-04
5,HbA1C%,"1.322 (1.117, 1.565)",1.178639e-03,"1.359 (1.186, 1.558)",9.848555e-06
6,Hcy,"1.055 (0.909, 1.225)",4.813007e-01,"1.012 (0.863, 1.186)",8.824440e-01
7,LDH,"1.012 (0.893, 1.146)",8.572904e-01,"0.906 (0.762, 1.077)",2.612680e-01
8,LDLC,"0.726 (0.636, 0.829)",2.302241e-06,"1.445 (1.252, 1.668)",5.053351e-07
9,LPa,"1.121 (0.861, 1.459)",3.963158e-01,"0.902 (0.789, 1.032)",1.334996e-01


## Supp Table 10

In [13]:
table_data = pd.read_csv(RawDataDir / "ClinicalModelPerformance.csv")
table_data

,DataSetName,score,AUC,AUC_UCI,AUC_LCI,ACC,ACC_UCI,ACC_LCI,Macro_F1,Macro_F1_UCI,...,Sensitivity_LCI,Specificity,Specificity_UCI,Specificity_LCI,APR,APR_UCI,APR_LCI,N,N_case,N_control
0,DX-EV,Clinical,0.613717,0.656375,0.570392,0.590327,0.628734,0.556188,0.589808,0.624415,...,0.490963,0.661017,0.716594,0.609648,0.675204,0.728106,0.627826,703.0,408.0,295.0
1,DX-EV,Predicted Clinical,0.649103,0.690248,0.609082,0.634424,0.667141,0.598862,0.631369,0.666462,...,0.576828,0.647458,0.703707,0.593557,0.677566,0.730343,0.626436,703.0,408.0,295.0
2,DX-EV,Predicted Clinical Full,0.656514,0.695673,0.617527,0.648649,0.687055,0.611664,0.637738,0.673434,...,0.666667,0.566102,0.622421,0.509745,0.698493,0.746996,0.646338,703.0,408.0,295.0


In [14]:
table_data["DataSetName"] = pd.Categorical(
    table_data["DataSetName"],
    # categories=["AZ-TR", "AZ-EV1", "AZ-EV2", "AZTZ-EV", "DX-EV"],
    categories=["AZ-TR", "DX-EV"],
    ordered=True,
)

table_data["score"] = pd.Categorical(
    table_data["score"],
    categories=["Clinical", "Predicted Clinical", "Predicted Clinical Full"],
    ordered=True,
)

In [15]:
# save to table


RenameDict = {
    "score": "Model",
    "Case_over_Control": "Case/Control",
}

table_df = table_data.copy()
add_metric = ["AUC", "ACC", "Macro_F1", "Sensitivity", "Specificity", "APR"]

for metric in add_metric:

    table_df[f"{metric} (95% CI)"] = format_metrics(
        table_df[f"{metric}"],
        x_lci=table_df[f"{metric}_LCI"],
        x_uci=table_df[f"{metric}_UCI"],
        round=3,
        data=table_df,
    )

table_df["Case_over_Control"] = (
    table_df["N_case"].astype(str) + "/" + table_df["N_control"].astype(str)
)
base_order = [
    "score",
    "DataSetName",
    "Case_over_Control",
]
table_df_format = (
    (
        table_df[base_order + [f"{metric} (95% CI)" for metric in add_metric]]
        .sort_values(
            [
                "score",
                "DataSetName",
            ]
        )
        .rename(columns=RenameDict)
    )
    .pivot(
        index="Model",
        columns="DataSetName",
        values=["Case/Control"] + [f"{metric} (95% CI)" for metric in add_metric],
    )
    .swaplevel(axis=1)
    .sort_index(axis=1)
)
table_df_format.to_excel(TableDir / "SupplementaryTable10.xlsx")
table_df_format

DataSetName                             DX-EV                        \
                                 ACC (95% CI)          APR (95% CI)   
Model                                                                 
Clinical                 0.590 (0.556, 0.629)  0.675 (0.628, 0.728)   
Predicted Clinical       0.634 (0.599, 0.667)  0.678 (0.626, 0.730)   
Predicted Clinical Full  0.649 (0.612, 0.687)  0.698 (0.646, 0.747)   

DataSetName                                                 \
                                 AUC (95% CI) Case/Control   
Model                                                        
Clinical                 0.614 (0.570, 0.656)  408.0/295.0   
Predicted Clinical       0.649 (0.609, 0.690)  408.0/295.0   
Predicted Clinical Full  0.657 (0.618, 0.696)  408.0/295.0   

DataSetName                                                          \
                            Macro_F1 (95% CI)  Sensitivity (95% CI)   
Model                                                                 
Clinical                 0.590 (0.554, 0.624)  0.539 (0.491, 0.589)   
Predicted Clinical       0.631 (0.594, 0.666)  0.625 (0.577, 0.668)   
Predicted Clinical Full  0.638 (0.602, 0.673)  0.708 (0.667, 0.752)   

DataSetName                                    
                         Specificity (95% CI)  
Model                                          
Clinical                 0.661 (0.610, 0.717)  
Predicted Clinical       0.647 (0.594, 0.704)  
Predicted Clinical Full  0.566 (0.510, 0.622)

## Supp Table 11

In [16]:
plt_data = pd.read_csv(RawDataDir / "RelativeImportance_Cardiovascular_full.csv")

In [20]:
T11 = plt_data[["Omics", "Importance_Real", "Importance_Predicted"]]
T11.to_excel(TableDir / "SupplementaryTable11.xlsx", index=False)
T11

,Omics,Importance_Real,Importance_Predicted
0,HDLC,0.231729,0.303351
1,HbA1C,0.169712,0.169296
2,Sex,0.131286,0.115455
3,BNP,0.094023,0.006040
4,TC,0.067828,0.041242
5,hs-CRP,0.057281,0.005575
6,nonHDL,0.046326,0.018002
7,LP(a),0.040840,0.022085
8,Height,0.034138,0.023628
9,hsTnI,0.022960,0.003599


## Supp Table 12 

In [41]:
ICC_res_df = pd.read_feather(RawDataDir / "ICC_res.feather")
ICC_res_df.shape

(744, 9)

In [48]:
ICC_res_df["Model"] = pd.Categorical(
    ICC_res_df["Model"], categories=["Ummae", "SwinLarge", "ResNet18"], ordered=True
)
ICC_res_df["Center"] = pd.Categorical(
    ICC_res_df["Center"],
    categories=["AZ-TR", "AZ-EV1", "AZ-EV2", "DX-EV"],
    ordered=True,
)

In [49]:
from ppp_prediction.metrics.utils import format_metrics

T12 = ICC_res_df.copy()
T12 = T12[
    [
        "Feature",
        "Model",
        "Center",
        "Mean ICC",
        "95% CI Lower",
        "95% CI Upper",
        "p_value",
    ]
].rename(
    columns={
        "Mean ICC": "ICC",
        "95% CI Lower": "ICC_LCI",
        "95% CI Upper": "ICC_UCI",
        "p_value": "Pval",
    }
)
for metric in ["ICC"]:
    T12[f"{metric} (95% CI)"] = format_metrics(
        T12[f"{metric}"],
        x_lci=T12[f"{metric}_LCI"],
        x_uci=T12[f"{metric}_UCI"],
        round=3,
        data=T12,
    )

T12 = T12[["Feature", "Model", "Center", "ICC (95% CI)", "Pval"]]
T12 = (
    T12.pivot(
        index="Feature",
        columns=["Center", "Model"],
        values=[
            "Pval",
            "ICC (95% CI)",
        ],
    )
    .sort_index(axis=1, level=[1, 2])
    .swaplevel(0, 2, axis=1)
    .swaplevel(0, 1, axis=1)
)
T12.to_excel(TableDir / "SupplementaryTable12.xlsx")
T12

Center                        AZ-TR                                  \
Model                         Ummae                  SwinLarge        
                       ICC (95% CI) Pval          ICC (95% CI) Pval   
Feature                                                               
AG             0.839 (0.830, 0.849)  0.0  0.800 (0.792, 0.807)  0.0   
ALP            0.884 (0.878, 0.890)  0.0  0.848 (0.844, 0.853)  0.0   
ALT            0.941 (0.939, 0.944)  0.0  0.893 (0.889, 0.897)  0.0   
APTT           0.899 (0.897, 0.901)  0.0  0.860 (0.854, 0.866)  0.0   
AST            0.850 (0.844, 0.856)  0.0  0.813 (0.804, 0.822)  0.0   
...                             ...  ...                   ...  ...   
eGFR(CKD-EPI)  0.961 (0.959, 0.963)  0.0  0.940 (0.936, 0.944)  0.0   
hs-CRP         0.825 (0.820, 0.830)  0.0  0.696 (0.676, 0.716)  0.0   
hsTnI          0.785 (0.779, 0.791)  0.0  0.414 (0.381, 0.447)  0.0   
nonHDL         0.800 (0.790, 0.811)  0.0  0.824 (0.815, 0.832)  0.0   
sdLDL          0.870 (0.862, 0.877)  0.0  0.776 (0.765, 0.788)  0.0   

Center                                                  AZ-EV1       \
Model                      ResNet18                      Ummae        
                       ICC (95% CI) Pval          ICC (95% CI) Pval   
Feature                                                               
AG             0.671 (0.659, 0.682)  0.0  0.784 (0.778, 0.791)  0.0   
ALP            0.487 (0.467, 0.507)  0.0  0.918 (0.916, 0.920)  0.0   
ALT            0.893 (0.888, 0.898)  0.0  0.949 (0.947, 0.951)  0.0   
APTT           0.477 (0.455, 0.500)  0.0  0.951 (0.950, 0.953)  0.0   
AST            0.722 (0.708, 0.736)  0.0  0.869 (0.865, 0.873)  0.0   
...                             ...  ...                   ...  ...   
eGFR(CKD-EPI)  0.895 (0.890, 0.901)  0.0  0.970 (0.968, 0.972)  0.0   
hs-CRP         0.479 (0.450, 0.508)  0.0  0.911 (0.908, 0.914)  0.0   
hsTnI          0.631 (0.591, 0.670)  0.0  0.889 (0.887, 0.892)  0.0   
nonHDL         0.717 (0.685, 0.749)  0.0  0.836 (0.832, 0.840)  0.0   
sdLDL          0.818 (0.803, 0.833)  0.0  0.885 (0.882, 0.888)  0.0   

Center                                          ...                  AZ-EV2  \
Model                      SwinLarge            ...               SwinLarge   
                        ICC (95% CI)      Pval  ...            ICC (95% CI)   
Feature                                         ...                           
AG              0.549 (0.543, 0.555)       0.0  ...    0.552 (0.548, 0.557)   
ALP             0.775 (0.770, 0.781)       0.0  ...    0.773 (0.771, 0.775)   
ALT             0.850 (0.848, 0.853)       0.0  ...    0.816 (0.814, 0.818)   
APTT            0.743 (0.739, 0.747)       0.0  ...    0.733 (0.729, 0.738)   
AST             0.317 (0.306, 0.328)       0.0  ...    0.373 (0.364, 0.382)   
...                              ...       ...  ...                     ...   
eGFR(CKD-EPI)   0.938 (0.937, 0.940)       0.0  ...    0.934 (0.933, 0.935)   
hs-CRP         0.006 (-0.003, 0.015)  0.685786  ...    0.030 (0.019, 0.042)   
hsTnI           0.022 (0.009, 0.036)  0.310271  ...  -0.009 (-0.024, 0.005)   
nonHDL          0.650 (0.645, 0.655)       0.0  ...    0.647 (0.641, 0.652)   
sdLDL           0.605 (0.597, 0.613)       0.0  ...    0.624 (0.619, 0.629)   

Center                                                     \
Model                                  ResNet18             
                   Pval            ICC (95% CI)      Pval   
Feature                                                     
AG                  0.0    0.261 (0.254, 0.267)       0.0   
ALP                 0.0    0.621 (0.613, 0.629)       0.0   
ALT                 0.0    0.878 (0.875, 0.881)       0.0   
APTT                0.0    0.160 (0.146, 0.174)       0.0   
AST                 0.0    0.259 (0.247, 0.271)       0.0   
...                 ...                     ...       ...   
eGFR(CKD-EPI)       0.0    0.927 (0.926, 0.928)       0.0   
hs-CRP     

## Supp Table 13 

In [51]:
T13 = ICC_res_df.copy()

T13 = T13[
    [
        "Feature",
        "Model",
        "Center",
        "Mean ICC",
        "95% CI Lower",
        "95% CI Upper",
        "p_value",
    ]
].rename(
    columns={
        "Mean ICC": "ICC",
        "95% CI Lower": "ICC_LCI",
        "95% CI Upper": "ICC_UCI",
        "p_value": "Pval",
    }
)
T13 = (
    T13.groupby(["Model", "Center"])
    .apply(lambda x: x["ICC"].describe()[["50%", "25%", "75%", "count"]])
    .sort_index()
)
T13[f"ICC (IQR)"] = (
    T13["50%"].apply(lambda x: f"{x:.3f}")
    + " ("
    + T13["25%"].apply(lambda x: f"{x:.3f}")
    + " - "
    + T13["75%"].apply(lambda x: f"{x:.3f}")
    + ")"
)
T13 = T13.reset_index(drop=False)[["Model", "Center", "ICC (IQR)"]].pivot(
    index="Model", columns="Center", values="ICC (IQR)"
)
T13.to_excel(TableDir / "SupplementaryTable13.xlsx")
T13

Center,AZ-TR,AZ-EV1,AZ-EV2,DX-EV
Model,,,,
Ummae,0.881 (0.841 - 0.933),0.915 (0.885 - 0.948),0.914 (0.882 - 0.947),0.972 (0.957 - 0.981)
SwinLarge,0.817 (0.706 - 0.897),0.721 (0.456 - 0.848),0.703 (0.465 - 0.828),0.870 (0.722 - 0.947)
ResNet18,0.800 (0.485 - 0.875),0.602 (0.199 - 0.835),0.606 (0.178 - 0.823),0.893 (0.536 - 0.945)


## Supp Table 14

In [78]:
qt_df = pd.read_feather(RawDataDir / "label_efficiency_qt.feather")
traits_used = qt_df["Trait"].unique().tolist()


totalDataDict = pd.read_pickle(RawDataDir / "totalDataDict.pkl")
tb1_df2 = (
    totalDataDict["Ummae"]
    .query("DataSetName == 'AZ-TR'")[["eid", "type", *traits_used]]
    .drop_duplicates()
)
need_counts = (~tb1_df2.query("type == 'train'")[[*traits_used]].isna()).sum().to_dict()
qt_df["Training_Sample_Size"] = qt_df.apply(
    lambda row: int(need_counts[row["Trait"]] * row["Ratio"]), axis=1
)
print(f"Training_Sample_Size max: {qt_df['Training_Sample_Size'].max()}")

bins = [0, 400, 800, 1600, 3300]  # as max is 3287
labels = [400, 800, 1600, 3200]
qt_df["Training_Sample_Size_bin"] = pd.cut(
    qt_df["Training_Sample_Size"], bins=bins, labels=labels
)
qt_df

Training_Sample_Size max: 3287


,Model,Trait,Center,N,Ratio,spearmanr,pearsonr,p_adj,pvalue,R2,R2_LCI,R2_UCI,R2 (95% CI),spearmanr_LCI,spearmanr_UCI,spearmanr (95% CI),pearsonr_LCI,pearsonr_UCI,pearsonr (95% CI),Training_Sample_Size,Training_Sample_Size_bin
0,ResNet18,AG,AZ-EV1,1498,0.2,-0.085819,0.023535,NaN,3.664222e-01,0.000554,-0.001871,0.002979,"0.001 (-0.002, 0.003)",NaN,NaN,None,NaN,NaN,None,632,800
1,ResNet18,AG,AZ-EV1,1498,0.1,-0.114163,0.006294,NaN,8.091697e-01,0.000040,-0.000609,0.000689,"0.000 (-0.001, 0.001)",NaN,NaN,None,NaN,NaN,None,316,400
2,ResNet18,AG,AZ-EV1,1498,0.5,-0.023961,0.003503,NaN,8.930760e-01,0.000012,-0.000349,0.000373,"0.000 (-0.000, 0.000)",NaN,NaN,None,NaN,NaN,None,1581,1600
3,ResNet18,ALP,AZ-EV1,1498,0.5,0.028192,0.098794,NaN,7.606305e-04,0.009760,-0.000327,0.019847,"0.010 (-0.000, 0.020)",NaN,NaN,None,NaN,NaN,None,1581,1600
4,ResNet18,ALP,AZ-EV1,1498,0.1,0.042453,0.082222,NaN,5.145877e-03,0.006760,-0.001660,0.015181,"0.007 (-0.002, 0.015)",NaN,NaN,None,NaN,NaN,None,316,400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3616,Ummae,Urea,DX-EV,895,1.0,0.189554,0.193474,1.176252e-08,3.793449e-09,0.037432,0.012629,0.062235,"0.037 (0.013, 0.062)",0.133096,0.235492,"0.190 (0.133, 0.235)",0.167554,0.252996,"0.193 (0.168, 0.253)",3169,3200
3617,Ummae,WBC,DX-EV,893,1.0,0.148479,0.139583,5.568552e-05,2.579895e-05,0.019483,0.001235,0.037732,"0.019 (0.001, 0.038)",0.102851,0.191618,"0.148 (0.103, 0.192)",0.085855,0.168623,"0.140 (0.086, 0.169)",2822,3200
3618,Ummae,Weight,DX-EV,800,1.0,0.682028,0.676797,2.055708e-147,1.063724e-148,0.458055,0.406409,0.509700,"0.458 (0.406, 0.510)",0.653147,0.719780,"0.682 (0.653, 0.720)",0.629848,0.738134,"0.677 (0.630, 0.738)",2832,3200
3619,Ummae,eGFR(CKD-EPI),DX-EV,895,1.0,0.474502,0.428795,1.041746e-44,1.153317e-45,0.183865,0.137257,0.230473,"0.184 (0.137, 0.230)",0.444578,0.515038,"0.475 (0.445, 0.515)",0.391157,0.480555,"0.429 (0.391, 0.481)",3169,3200


In [79]:
qt_df["Trait"].unique()

array(['AG', 'ALP', 'ALT', 'APTT', 'AST', 'Age', 'Alb', 'BMI', 'BNP',
       'C1q', 'CK', 'CK-MB', 'Cl', 'Cr', 'D-Bil', 'D-Dimer', 'FBG', 'FFA',
       'FT3', 'FT4', 'GGT', 'Glu', 'HCT', 'HDLC', 'Hb', 'HbA1C%', 'Hcy',
       'Height', 'INR', 'Isn', 'K', 'LDH', 'LDLC', 'LPa', 'LYM', 'LYM%',
       'MCH', 'MCHC', 'MCV', 'MPV', 'NE', 'NE%', 'Na', 'PCT', 'PLT',
       'RBC', 'T-Bil', 'T3', 'T4', 'TC', 'TG', 'TP', 'TSH', 'UA', 'Urea',
       'WBC', 'Weight', 'eGFR(CKD-EPI)', 'hs-CRP', 'hsTnI', 'nonHDL',
       'sdLDL'], dtype=object)

In [80]:
Trait_order = [
    "Age",
    "ALP",
    "ALT",
    "APTT",
    "AST",
    "Alb",
    "AG",
    "BMI",
    "BNP",
    "C1q",
    "CK",
    "CK-MB",
    "Cl",
    "Cr",
    "D-Bil",
    "D-Dimer",
    "EF",
    "FBG",
    "FFA",
    "FT3",
    "FT4",
    "GGT",
    "Glu",
    "HCT",
    "HDLC",
    "Hb",
    "HbA1C%",
    "Hcy",
    "Height",
    "INR",
    "Isn",
    "K",
    "LDH",
    "LDLC",
    "LPa",
    "LYM",
    "LYM%",
    "MCH",
    "MCHC",
    "MCV",
    "MPV",
    "NE",
    "NE%",
    "Na",
    "PCT",
    "PLT",
    "RBC",
    "T-Bil",
    "T3",
    "T4",
    "TC",
    "TG",
    "TP",
    "TSH",
    "UA",
    "Urea",
    "WBC",
    "Weight",
    "eGFR(CKD-EPI)",
    "hs-CRP",
    "hsTnI",
    "nonHDL",
    "sdLDL",
]

In [84]:
qt_df.query("Model =='Ummae' and Trait =='nonHDL'")

,Model,Trait,Center,N,Ratio,spearmanr,pearsonr,p_adj,pvalue,R2,R2_LCI,R2_UCI,R2 (95% CI),spearmanr_LCI,spearmanr_UCI,spearmanr (95% CI),pearsonr_LCI,pearsonr_UCI,pearsonr (95% CI),Training_Sample_Size,Training_Sample_Size_bin
552,Ummae,nonHDL,AZ-EV1,1498,0.5,0.100509,0.135309,NaN,2.149881e-07,0.018309,0.004613,0.032005,"0.018 (0.005, 0.032)",NaN,NaN,None,NaN,NaN,None,1605,3200
553,Ummae,nonHDL,AZ-EV1,1498,0.2,0.107091,0.119545,NaN,4.822438e-06,0.014291,0.002141,0.026441,"0.014 (0.002, 0.026)",NaN,NaN,None,NaN,NaN,None,642,800
554,Ummae,nonHDL,AZ-EV1,1498,0.1,0.093250,0.081502,NaN,1.901125e-03,0.006643,-0.001705,0.014990,"0.007 (-0.002, 0.015)",NaN,NaN,None,NaN,NaN,None,321,400
1110,Ummae,nonHDL,AZ-EV2,2247,0.2,0.131871,0.155483,NaN,5.410946e-13,0.024175,0.011392,0.036958,"0.024 (0.011, 0.037)",NaN,NaN,None,NaN,NaN,None,642,800
1111,Ummae,nonHDL,AZ-EV2,2247,0.1,0.128090,0.138634,NaN,1.394620e-10,0.019219,0.007764,0.030675,"0.019 (0.008, 0.031)",NaN,NaN,None,NaN,NaN,None,321,400
1112,Ummae,nonHDL,AZ-EV2,2247,0.5,0.112216,0.136054,NaN,3.078307e-10,0.018511,0.007260,0.029761,"0.019 (0.007, 0.030)",NaN,NaN,None,NaN,NaN,None,1605,3200
1668,Ummae,nonHDL,AZ-TR,480,0.2,0.142772,0.125173,NaN,6.179528e-03,0.015668,-0.006664,0.038000,"0.016 (-0.007, 0.038)",NaN,NaN,None,NaN,NaN,None,642,800
1669,Ummae,nonHDL,AZ-TR,480,0.5,0.122890,0.105335,NaN,2.151472e-02,0.011095,-0.007784,0.029975,"0.011 (-0.008, 0.030)",NaN,NaN,None,NaN,NaN,None,1605,3200
1670,Ummae,nonHDL,AZ-TR,480,0.1,0.081829,0.081251,NaN,7.685911e-02,0.006602,-0.008028,0.021231,"0.007 (-0.008, 0.021)",NaN,NaN,None,NaN,NaN,None,321,400
2226,Ummae,nonHDL,AZTZ-EV,206,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"nan (nan, nan)",NaN,NaN,None,NaN,NaN,None,321,400


In [ ]:
pd.set_option("display.max_columns", None)
from ppp_prediction.metrics.utils import format_metrics

T14 = qt_df.copy().dropna(subset=["R2"])
T14["Trait"] = pd.Categorical(T14["Trait"], Trait_order, ordered=True)
add_metric = ["R2"]

for metric in add_metric:

    T14[f"{metric} (95% CI)"] = format_metrics(
        T14[f"{metric}"],
        x_lci=T14[f"{metric}_LCI"],
        x_uci=T14[f"{metric}_UCI"],
        round=3,
        data=T14,
    )

T14 = (
    T14[
        [
            "Trait",
            "Model",
            "Center",
            "Training_Sample_Size",
            "Ratio",
            "N",
            "R2 (95% CI)",
        ]
    ]
    .rename(
        columns={
            "Training_Sample_Size": "Train sample size",
            "N": "Test sample size",
        }
    )
    .reset_index(drop=True)
)
T14["Center"] = pd.Categorical(
    T14["Center"],
    categories=["AZ-TR", "AZ-EV1", "AZ-EV2", "AZTZ-EV", "DX-EV"],
    ordered=True,
)
T14["Model"] = pd.Categorical(
    T14["Model"], categories=["Ummae", "SwinLarge", "ResNet18"], ordered=True
)
# T14["Ratio"] = pd.Categorical(
#     T14["Ratio"], categories=["10%", "20%", "50%", "100%"], ordered=True

# T14["Train sample size"] = pd.Categorical(
#     T14["Train sample size"], categories=[400, 800, 1600, 3200], ordered=True
# )

T14_format = (
    T14.sort_values(["Trait", "Model", "Center"])
    .reset_index(drop=True)
    .pivot(
        index=[
            "Trait",
            # "Ratio",
            "Train sample size",
        ],
        values=[
            "R2 (95% CI)",
            "Test sample size",
        ],
        columns=[
            "Model",
            "Center",
        ],
    )
    .swaplevel(
        0,
        2,
        axis=1,
    )
    .sort_index(axis=1, level=(0, 1))
)

T14_format = T14_format.sort_index()
import numpy as np

T14_format = T14_format.replace(np.nan, "\\")
T14_format.to_excel(TableDir / "Supplementary Table14.xlsx")
T14_format

Center                                    AZ-TR                   \
Model                                     Ummae                    
                                    R2 (95% CI) Test sample size   
Trait  Train sample size                                           
Age    328                 0.819 (0.789, 0.848)              480   
       657                 0.839 (0.812, 0.865)              480   
       1643                0.863 (0.840, 0.886)              480   
       3287                0.873 (0.852, 0.895)              480   
ALP    316                0.002 (-0.007, 0.011)              480   
...                                         ...              ...   
nonHDL 3210                0.038 (0.004, 0.072)              473   
sdLDL  261                0.015 (-0.007, 0.038)              480   
       523                 0.043 (0.007, 0.079)              480   
       1309                0.041 (0.006, 0.077)              480   
       2618               0.037 (-0.001, 0.074)              387   

Center                                                            \
Model                                 SwinLarge                    
                                    R2 (95% CI) Test sample size   
Trait  Train sample size                                           
Age    328                 0.780 (0.745, 0.816)              480   
       657                 0.812 (0.781, 0.842)              480   
       1643                0.850 (0.824, 0.875)              480   
       3287                0.870 (0.848, 0.892)              480   
ALP    316                0.003 (-0.007, 0.014)              480   
...                                         ...              ...   
nonHDL 3210               0.025 (-0.003, 0.053)              473   
sdLDL  261                0.001 (-0.005, 0.008)              480   
       523                0.001 (-0.004, 0.005)              480   
       1309               0.014 (-0.007, 0.035)              480   
       2618               0.034 (-0.002, 0.070)              387   

Center                                                            \
Model                                  ResNet18                    
                                    R2 (95% CI) Test sample size   
Trait  Train sample size                                           
Age    328                 0.776 (0.740, 0.812)              480   
       657                 0.809 (0.778, 0.841)              480   
       1643                0.826 (0.797, 0.855)              480   
       3287                0.854 (0.829, 0.878)              480   
ALP    316                0.001 (-0.005, 0.008)              480   
...                                         ...              ...   
nonHDL 3210               0.007 (-0.008, 0.022)              473   
sdLDL  261                0.015 (-0.007, 0.037)              480   
       523                0.003 (-0.007, 0.013)              480   
       1309               0.003 (-0.007, 0.012)              480   
       2618               0.004 (-0.009, 0.017)              387   

Center                                  AZ-EV1                   \
Model                                    Ummae                    
                                   R2 (95% CI) Test sample size   
Trait  Train sample size                                          
Age    328                0.802 (0.783, 0.820)             1498   
       657                0.823 (0.806, 0.839)             1498   
       1643               0.851 (0.837, 0.865)             1498   
       3287               0.863 (0.850, 0.876)             1498   
ALP    316                0.019 (0.005, 0.033)             1498   
...                                        ...              ...   
nonHDL 3210               0.036 (0.016, 0.055)             1444   
sdLDL  261                0.021 (0.006, 0.036)             1498   
       523                0.056 (0.033, 0.079)             1498   
       1309               0.057 (0.034, 0.081)             1498   

Center                                    AZ-TR                   \
Model                                     Ummae                    
                                    R2 (95% CI) Test sample size   
Trait  Train sample size                                           
Age    328                 0.819 (0.789, 0.848)              480   
       657                 0.839 (0.812, 0.865)              480   
       1643                0.863 (0.840, 0.886)              480   
       3287                0.873 (0.852, 0.895)              480   
ALP    316                0.002 (-0.007, 0.011)              480   
...                                         ...              ...   
nonHDL 3210                0.038 (0.004, 0.072)              473   
sdLDL  261                0.015 (-0.007, 0.038)              480   
       523                 0.043 (0.007, 0.079)              480   
       1309                0.041 (0.006, 0.077)              480   
       2618               0.037 (-0.001, 0.074)              387   

Center                                                            \
Model                                 SwinLarge                    
                                    R2 (95% CI) Test sample size   
Trait  Train sample size                                           
Age    328                 0.780 (0.745, 0.816)              480   
       657                 0.812 (0.781, 0.842)              480   
       1643                0.850 (0.824, 0.875)              480   
       3287                0.870 (0.848, 0.892)              480   
ALP    316                0.003 (-0.007, 0.014)              480   
...                                         ...              ...   
nonHDL 3210               0.025 (-0.003, 0.053)              473   
sdLDL  261                0.001 (-0.005, 0.008)              480   
       523                0.001 (-0.004, 0.005)              480   
       1309               0.014 (-0.007, 0.035)              480   
       2618               0.034 (-0.002, 0.070)              387   

Center                                                            \
Model                                  ResNet18                    
                                    R2 (95% CI) Test sample size   
Trait  Train sample size                                           
Age    328                 0.776 (0.740, 0.812)              480   
       657                 0.809 (0.778, 0.841)              480   
       1643                0.826 (0.797, 0.855)              480   
       3287                0.854 (0.829, 0.878)              480   
ALP    316                0.001 (-0.005, 0.008)              480   
...                                         ...              ...   
nonHDL 3210               0.007 (-0.008, 0.022)              473   
sdLDL  261                0.015 (-0.007, 0.037)              480   
       523                0.003 (-0.007, 0.013)              480   
       1309               0.003 (-0.007, 0.012)              480   
       2618               0.004 (-0.009, 0.017)              387   

Center                                  AZ-EV1                   \
Model                                    Ummae                    
                                   R2 (95% CI) Test sample size   
Trait  Train sample size                                          
Age    328                0.802 (0.783, 0.820)             1498   
       657                0.823 (0.806, 0.839)             1498   
       1643               0.851 (0.837, 0.865)             1498   
       3287               0.863 (0.850, 0.876)             1498   
ALP    316                0.019 (0.005, 0.033)             1498   
...                                        ...              ...   
nonHDL 3210               0.036 (0.016, 0.055)             1444   
sdLDL  261                0.021 (0.006, 0.036)             1498   
       523                0.056 (0.033, 0.079)             1498   
       1309               0.057 (0.034, 0.081)             1498   

## Supp Table 16

In [35]:
compare_res_df_fdr = pd.read_feather(RawDataDir / "Phone_qt_df.feather")
T16 = compare_res_df_fdr[["Traits", "N", "p_adj", "R2 (95% CI)"]].sort_values("Traits")
T16.to_excel(TableDir / "SupplementaryTable16.xlsx", index=False)
T16

,Traits,N,p_adj,R2 (95% CI)
54,AG,110.0,7.911484e-01,"0.001 (-0.010, 0.012)"
6,ALP,54.0,7.889712e-04,"0.221 (0.034, 0.407)"
20,ALT,107.0,9.326391e-02,"0.044 (-0.031, 0.119)"
46,APTT,59.0,7.099436e-01,"0.004 (-0.027, 0.035)"
22,AST,108.0,1.444579e-01,"0.034 (-0.032, 0.100)"
0,Age,110.0,2.998469e-86,"0.786 (0.715, 0.856)"
45,Alb,51.0,7.099436e-01,"0.005 (-0.031, 0.041)"
3,BMI,106.0,1.587715e-14,"0.382 (0.238, 0.525)"
42,CK,106.0,6.341227e-01,"0.006 (-0.023, 0.035)"
34,CK-MB,107.0,4.970435e-01,"0.011 (-0.028, 0.051)"


## Supp Table 17 

In [37]:
ICC_df_by_center = pd.read_feather(RawDataDir / "ICC_df_by_center_APP.feather")
T17 = ICC_df_by_center[["ComapreCenter", "ICC 95% CI", "pval"]]

T17.to_excel(TableDir / "SupplementaryTable17.xlsx")
T17

,ComapreCenter,ICC 95% CI,pval
0,AZ-TR,"0.88 (0.81, 0.93)",0.0
1,AZ-EV1,"0.91 (0.85, 0.94)",0.0
2,AZ-EV2,"0.94 (0.90, 0.96)",0.0
3,DX-EV,"0.94 (0.88, 0.97)",0.0
4,AZTZ-EV,"0.90 (0.83, 0.94)",0.0
